# Fourier Shell Correlation I : Indexing

Developing code for Fourier Shell Correlation for a project inspired by a recent paper on 3D resolution evaluation in CryoEM.

*Tan, Yong Zi, Philip R Baldwin, Joseph H Davis, James R Williamson, Clinton S Potter, Bridget Carragher, and Dmitry Lyumkis. “Addressing Preferred Specimen Orientation in Single-Particle Cryo-EM through Tilting.” Nature Methods 14, no. 8 (July 3, 2017): 793–96. doi:10.1038/nmeth.4347.*

In the paper they introduce a method that is able to deal with anisotropic resolution. For this purpose they modified the traditional FSC, by introducing an inclinatino angle term. 

In [1]:
import numpy as np
from mayavi import mlab
#mlab.init_notebook(local=False)

from supertomo.data.iterators.fourier_shell_iterators import ConicalFourierShellIterator
#from supertomo.data.iterators.fourier_shell_correlation import DirectionalFSC
from supertomo.data.containers.image import Image

## Regular spherical iteration

The authors provided their implementation of the algorithm no GitHub. However, that was quite horrible and I decided to rewrite the whole thing, based on my earlier FRC implementation. The first step isto figure out the indexing, as in order to add the angle term $\phi$ for changing the orientation of resolution calculation, I needed to create an iterator that can work with spherical coordinates. Here I'll test drive it with some simple test data.

In [2]:
#Create a test dataset and iterator
dataset = Image(np.ones((255,255,255), dtype=np.uint8)*100, (0.05,0.05,0.05))
iterator = ConicalFourierShellIterator(dataset.shape, 6, 30)

#Extract a section (bin_start, bin_stop, angle_start, angle_stop)
section = iterator[(100, 106, 30, 60)]
dataset[section] = 255

#Show
mlab.contour3d(dataset)
mlab.show()

## Hollow section iterator